In [ ]:
import django
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'gpw.settings'
django.setup()
from gpw.models import Statistics, Company
import pandas as pd
%matplotlib notebook

In [ ]:
from itertools import groupby
from operator import attrgetter
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def create_companies_df():
    companies = Company.objects.all().values()
    df = pd.DataFrame.from_records(companies)
    df.drop(['id'], inplace=True, axis=1)
    return df

def create_statistics_df():
    statistics = Statistics.objects.all().values()
    df = pd.DataFrame.from_records(statistics)
    df.drop(['id'], inplace=True, axis=1)
    columns_to_drop = []
    for column in df:
        unique_values = df[column].unique()
        if len(unique_values) == 1:
            columns_to_drop.append(column)
    print('Dropping columns from statistics: {}'.format(columns_to_drop))
    df.drop(columns_to_drop, inplace=True, axis=1)
    return df

def plot_company_stats(stats_df, company, stats_column='close_price'):
    company_stats = stats_df.loc[stats_df['isin'] == company['isin']]
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y/%m/%d'))
    plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
    plt.plot(company_stats['date'], company_stats[stats_column], label=company['name'])
    plt.gcf().autofmt_xdate()
    
def plot_sector_stats(stats_df, companies_df, sector):
    companies_in_sector = companies_df.loc[companies_df['sector'] == sector]
    for _, company in companies_in_sector.iterrows():
        plot_company_stats(stats_df, company)
    ax = plt.subplot(111)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
companies_df = create_companies_df()
statistics_df = create_statistics_df()

In [ ]:
plot_sector_stats(statistics_df, companies_df, 'Media')